# Analyzing networks of characters in _Love Actually_ in Python

Source material in R: http://varianceexplained.org/r/love-actually-network/

In [1]:
!which python

/cygdrive/c/users/narho_000/anaconda3/python
